In [1]:
using LinearAlgebra
using LoopVectorization
using LinearMaps
using ArnoldiMethod

function prepareDiag(L)
    diag = zeros(2^L)
    for state = 1:2^L
        for i = 1:L
            j = i==L ? 1 : i+1
            @inbounds diag[state] -= (((state >> (i-1))&1) == ((state >> (j-1))&1)) ? 1 : -1
        end
    end
    diag
end
    
function Hfunc!(C, B, diag, L)
    N = length(diag)
    @tturbo for state = 1:N
        C[state] = diag[state] * B[state]
    end
    for i = 1:L
        @tturbo for state = 1:N
            newstate = (state&(~(2^L))) ⊻ (1<<(i-1))
            c = newstate == 0
            newstate = !c*newstate + c*N # remove if statement
            C[newstate] -= B[state]
        end
    end
end
prepareHfunc!(L) = (C, B) -> Hfunc!(C, B, prepareDiag(L), L)

function f_LinearMap(L)
    H! = prepareHfunc!(L)
    H_LinearMap = LinearMap(H!, 2^L, ismutating=true, issymmetric=true, isposdef=false)
    d_LM, h_LM = partialschur(H_LinearMap, nev=1, which=SR())
    d_LM.eigenvalues[1]
end

f_exact(L) = -2sum(abs(sin((n-1/2) * pi/L)) for n in 1:L)

f_exact (generic function with 1 method)

In [7]:
# https://juliaphysics.github.io/PhysicsTutorials.jl/tutorials/general/quantum_ising/quantum_ising.html

using SparseArrays
using ArnoldiMethod
⊗(x,y) = kron(x,y)

function TransverseFieldIsing_sparse(;N,h)
    id = [1 0; 0 1] |> sparse
    σˣ = [0 1; 1 0] |> sparse
    σᶻ = [1 0; 0 -1] |> sparse
    
    first_term_ops = fill(id, N)
    first_term_ops[1] = σᶻ
    first_term_ops[2] = σᶻ
    
    second_term_ops = fill(id, N)
    second_term_ops[1] = σˣ
    
    H = spzeros(Int, 2^N, 2^N) # note the spzeros instead of zeros here
    #for i in 1:N-1
    for i in 1:N # periodic boundary condition
        H -= foldl(⊗, first_term_ops)
        first_term_ops = circshift(first_term_ops,1)
    end
    
    for i in 1:N
        H -= h*foldl(⊗, second_term_ops)
        second_term_ops = circshift(second_term_ops,1)
    end
    H
end

function f_sparse(L)
    H = TransverseFieldIsing_sparse(N=L, h=1)
    d, h = partialschur(H; nev=1, which=SR())
    d.eigenvalues[1]
end

f_sparse (generic function with 1 method)

In [3]:
f_exact(20)

-25.49098968636475

In [4]:
@time f_LinearMap(20)
@time f_LinearMap(20)

  8.176899 seconds (92.59 k allocations: 973.226 MiB, 3.00% gc time)
  7.878416 seconds (620 allocations: 968.031 MiB, 0.44% gc time)


-25.490989686364784 + 0.0im

In [5]:
@time f_sparse(20)
@time f_sparse(20)

 10.483391 seconds (4.72 k allocations: 7.038 GiB, 6.81% gc time)
 10.396778 seconds (4.72 k allocations: 7.038 GiB, 5.98% gc time)


-25.49098968636471 + 0.0im

In [6]:
@time H = TransverseFieldIsing_sparse(N=20, h=1)
@time d, h = partialschur(H; nev=1, which=SR())

  3.827107 seconds (4.66 k allocations: 6.718 GiB, 22.51% gc time)
  7.067865 seconds (248 allocations: 328.026 MiB, 1.34% gc time, 0.23% compilation time)


(ArnoldiMethod.PartialSchur{SubArray{Float64, 2, Matrix{Float64}, Tuple{Base.Slice{Base.OneTo{Int64}}, UnitRange{Int64}}, true}, SubArray{Float64, 2, Matrix{Float64}, Tuple{UnitRange{Int64}, UnitRange{Int64}}, false}, ComplexF64}([0.23608646324993104; 0.06481741688770436; … ; 0.06481741672801705; 0.2360864632028416;;], [-25.490989686364795;;], ComplexF64[-25.490989686364795 + 0.0im]), Converged: 1 of 1 eigenvalues in 80 matrix-vector products)